In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

* Update 06/01/2024: Many thanks to our soon-to-be Honor Student **Nicolás Oliveira** for the suggestions to update this code to the lastest LangChain version!

### Connect with LM

In [2]:
from langchain_openai import ChatOpenAI

In [3]:
my_creative_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.9)

In [4]:
question = "What are the 5 best things to do in life?"

In [5]:
print(my_creative_llm.invoke(question))

content="1. Travel and explore new places: Traveling allows you to experience new cultures, meet new people, and create lasting memories. It can broaden your perspectives and help you grow as a person.\n\n2. Pursue your passions and hobbies: Engaging in activities that you love can bring joy, fulfillment, and a sense of purpose to your life. Whether it's painting, playing a musical instrument, cooking, or hiking, make time for the things that make you happy.\n\n3. Build meaningful relationships: Cultivating strong, healthy relationships with family and friends is essential for overall well-being. Spend quality time with loved ones, listen actively, and show kindness and appreciation towards them.\n\n4. Practice self-care and prioritize your health: Taking care of yourself physically, mentally, and emotionally is crucial for a fulfilling life. Eat well, exercise regularly, get enough sleep, and manage stress effectively. Make time for activities that promote relaxation and self-reflecti

### Create a prompt template to wrap the user input

In [6]:
from langchain_core.prompts import PromptTemplate

In [7]:
my_prompt_template = PromptTemplate(
    input_variables = ["destination"],
    template = "What are the 3 best things to do in {destination}?"
)

In [8]:
user_input = "Barcelona"

In [9]:
print(my_creative_llm.invoke(my_prompt_template.format(destination=user_input)))

content="1. Visit the famous Sagrada Familia: This iconic basilica, designed by architect Antoni Gaudí, is a must-see attraction in Barcelona. Its intricate façade and towering spires make it a breathtaking sight to behold.\n\n2. Explore the Gothic Quarter: Wander through the narrow cobblestone streets of this historic neighborhood, filled with medieval buildings, charming plazas, and hidden gems. Stop by the Barcelona Cathedral and the Picasso Museum for a dose of culture and history.\n\n3. Relax at the beach: Barcelona boasts several beautiful beaches along its coastline, including Barceloneta Beach, where you can soak up the sun, take a dip in the Mediterranean Sea, or enjoy a leisurely stroll along the promenade. Don't forget to try some fresh seafood at one of the beachfront restaurants."


### Combine instructions in chains

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

In [11]:
my_no_creative_llm = ChatOpenAI(temperature=0)

In [12]:
my_prompt_template = PromptTemplate(
    input_variables = ["politician"],
    template = "What are the 3 most remarkable achievements of {politician}?",
)

In [13]:
my_chain = LLMChain(llm=my_no_creative_llm, prompt=my_prompt_template)

In [23]:
print(my_chain.invoke("Churchill"))

{'politician': 'Churchill', 'text': '1. Leadership during World War II: Winston Churchill\'s leadership as Prime Minister of the United Kingdom during World War II was one of his most remarkable achievements. He inspired the British people to resist Nazi Germany and played a key role in the Allied victory over the Axis powers.\n\n2. Iron Curtain speech: In 1946, Churchill delivered his famous "Iron Curtain" speech in Fulton, Missouri, in which he warned of the growing threat of Soviet expansionism in Eastern Europe. This speech is considered a turning point in the Cold War and helped to shape Western policy towards the Soviet Union.\n\n3. Nobel Prize in Literature: In 1953, Churchill was awarded the Nobel Prize in Literature for his historical writings and speeches, particularly his accounts of World War II. This recognition solidified his reputation as one of the greatest statesmen and orators of the 20th century.'}


### Agents: give reasoning power to LMs
1. Load some external tools (google search, llm-math, etc)
2. Initialize an agent type on top of the LM
3. Run the agent and she will decide the tools to use based on your input

*Careful: operations with agents are more expensive in tokens. We will not execute the code in this demo. To comment out an entire cell in Mac: Command + /*

In [15]:
# from langchain.agents import load_tools
# from langchain.agents import initialize_agent
# from langchain.llms import OpenAI
# tools = load_tools(["serpapi", "llm-math"], llm=llm)
# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
# agent.run("Who is the current leader of Japan? What is the largest prime number that is smaller than their age?")

### Add memory

In [16]:
from langchain import ConversationChain

In [17]:
my_no_creative_llm = ChatOpenAI(temperature=0)

In [18]:
my_conversation = ConversationChain(llm=my_no_creative_llm, verbose=True)

In [19]:
my_conversation.invoke("hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi there!
AI:

> Finished chain.


{'input': 'hi there!',
 'history': '',
 'response': 'Hello! How can I assist you today?'}

In [20]:
my_conversation.invoke("My name is Julio. What is your name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi there!
AI: Hello! How can I assist you today?
Human: My name is Julio. What is your name?
AI:

> Finished chain.


{'input': 'My name is Julio. What is your name?',
 'history': 'Human: hi there!\nAI: Hello! How can I assist you today?',
 'response': 'Hello Julio! My name is AI-9000. How can I help you today?'}

In [21]:
my_conversation.predict(input="Do you remember my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi there!
AI: Hello! How can I assist you today?
Human: My name is Julio. What is your name?
AI: Hello Julio! My name is AI-9000. How can I help you today?
Human: Do you remember my name?
AI:

> Finished chain.


'Yes, Julio! I have the ability to remember details like your name to provide personalized assistance. How can I assist you further today?'

In [22]:
print(my_conversation)

memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='hi there!'), AIMessage(content='Hello! How can I assist you today?'), HumanMessage(content='My name is Julio. What is your name?'), AIMessage(content='Hello Julio! My name is AI-9000. How can I help you today?'), HumanMessage(content='Do you remember my name?'), AIMessage(content='Yes, Julio! I have the ability to remember details like your name to provide personalized assistance. How can I assist you further today?')])) verbose=True llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10f70a410>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10f0365d0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')
